In [2]:
import numpy as np
import pandas as pd
import json
import gzip

from os import walk
from scipy.spatial import KDTree

In [3]:
LBPath = r'C:\Users\marvi\Desktop\SchemaOrgData\LocalBusiness_top100' + '\\'
LBFilenames = next(walk(LBPath), (None, None, []))[2]
LBData = []

for lb in LBFilenames[:20]:
    with gzip.open(LBPath + lb, 'r') as dataFile:
        for line in dataFile:
            lineData = json.loads(line.decode('utf-8'))
            LBData.append(lineData)
lbDF = pd.DataFrame(LBData)


In [4]:
# convert Geo into two columns (long + lat)
geoData = lbDF[lbDF['geo'].notnull()]
data = pd.concat([geoData, geoData['geo'].apply(pd.Series)], axis=1)

In [27]:
# Cut out entries that only have one geo-value
# Very few entrys only have either longitude or latitude
data = data[data['longitude'].notna()]
data = data[data['latitude'].notna()]

In [28]:
#pip install geopy
#replace , with . to solve Formation issues
lon = 'longitude'
lat = 'latitude'
data[lon] = data[lon].str.replace(',','.')
data[lat] = data[lat].str.replace(',','.')


In [55]:
from scipy.spatial import KDTree

def createKDTree(tupleArray):
    tree = KDTree(tupleArray)
    return tree

# Return all values that are in a specific proximity
def queryTree(tree, point, neighbours = 20):    
    point = [float(i) for i in point]
    #idx = tree.query_ball_point(point, r=0)
    #return idx
    idx = tree.query(point, k=neighbours)
    return idx[1]

# convert to tuples and from string to float
lonArr = data[lon].to_numpy()
latArr = data[lat].to_numpy()
tupleArray = np.array((lonArr, latArr)).T.astype('float32')

# create new column with all matching points
tree = createKDTree(tupleArray)
idx = queryTree(tree, tupleArray[0])

# Search for the closest neighbour in all of the points
data['MatchingPoints'] = data.apply(lambda row: queryTree(tree,[row[lon], row[lat]]), axis=1) 



In [52]:
data['MatchingPoints'][:100]

21373    []
21375    []
21380    []
21382    []
21383    []
         ..
21505    []
21506    []
21507    []
21510    []
21511    []
Name: MatchingPoints, Length: 100, dtype: object

In [31]:
# install if necessary
# pip install geopy

In [32]:
import geopy.distance

def calcDifference(pointOne, pointTwo):
    return geopy.distance.great_circle(pointOne, pointTwo).km

In [33]:
data['Distance'] = data['MatchingPoints'].apply(lambda row: calcDifference( (data.iloc[row[0]][lat], data.iloc[row[0]][lon]),(data.iloc[row[1]][lat], data.iloc[row[1]][lon])))
data['Distance'] = pd.to_numeric(data['Distance'])

In [34]:
data['Distance'].dtypes

dtype('float64')

In [35]:
one = 0
two = 1

pointOne = (data.iloc[one][lat], data.iloc[one][lon])
pointTwo = (data.iloc[two][lat], data.iloc[two][lon])

calcDifference(pointOne, pointTwo)

# Result = Distance Bielefeld - Leipzig 

274.567370042535

In [38]:
data[data['Distance'] < 0.005]

,row_id,name,address,telephone,page_url,aggregaterating,email,geo,url,image,...,3,4,5,6,7,8,latitude,longitude,MatchingPoints,Distance
21400,27,Apollo-Optik,"{'addressregion': 'Rheinland-Pfalz', 'addressc...",(06131)&nbsp;693876,https://www.11880.com/branchenbuch/mainz-am-rh...,"{'reviewcount': '12', 'bestrating': '5', 'wors...",service@apollo-optik.com,"{'latitude': '50,001', 'longitude': '8,270224'}",https://www.apollo.de/store-7053/mainz,https://cdn.11880.com/apollo-optik_17032642_mw...,...,NaN,NaN,NaN,NaN,NaN,NaN,50.001,8.270224,"[12, 10272]",0.000000
21451,78,iDT Ihr-Dienstleistungs-Team Region Dortmund,"{'streetaddress': 'Ruhrallee 9', 'postalcode':...",None,https://www.11880.com/branchenbuch/dortmund/06...,None,thimma@idt-online.de,"{'latitude': '51,508139', 'longitude': '7,4675...",https://www.idt-online.de,//static.11880.com/Portal/images/11880/photo-u...,...,NaN,NaN,NaN,NaN,NaN,NaN,51.508139,7.467584,"[47, 1640]",0.000000
21464,91,Ralf und Doris Aschauer Montageservice + Innen...,"{'addressregion': 'Baden-Württemberg', 'addres...",(07127)&nbsp;237871,https://www.11880.com/branchenbuch/neckartailf...,None,None,"{'longitude': '9,270464', 'latitude': '48,6110...",https://www.dvag.de,//static.11880.com/Portal/images/11880/photo-u...,...,NaN,NaN,NaN,NaN,NaN,NaN,48.611063,9.270464,"[58, 8703]",0.000000
21484,111,Dr.med. Oliver M. Mainusch Facharzt für Dermat...,"{'addresslocality': 'Velbert', 'addresscountry...",(02051)&nbsp;57783,https://www.11880.com/branchenbuch/velbert/110...,"{'reviewcount': '2', 'bestrating': '5', 'worst...",None,"{'longitude': '7,04722', 'latitude': '51,338768'}",http://www.hautconnection.de,//static.11880.com/Portal/images/11880/photo-u...,...,NaN,NaN,NaN,NaN,NaN,NaN,51.338768,7.04722,"[76, 2399]",0.000000
21511,138,Anwaltskanzlei Linden und Mosel Rechtsanwälte,"{'addresscountry': 'DE', 'addresslocality': 'K...",(0221)&nbsp;422220,https://www.11880.com/branchenbuch/koeln/08131...,"{'ratingvalue': '5.00', 'bestrating': '5', 'wo...",info@lindenundmosel.de,"{'longitude': '6,923274', 'latitude': '50,9229...",https://lindenundmosel.de,https://cdn.11880.com/anwaltskanzlei-linden-un...,...,NaN,NaN,NaN,NaN,NaN,NaN,50.922934,6.923274,"[99, 120]",0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203042,7048,"Baja . Gospodarstwo agroturystyczne, sklep","{'addresslocality': 'Swornegacie', 'streetaddr...",52 398 11 71,https://www.biznesfinder.pl/pomorskie%2C%20s%C...,None,baja55@poczta.onet.pl,"{'latitude': '53.8596725490196', 'longitude': ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,53.8596725490196,17.4955469411765,"[89946, 86574]",0.003986
203063,7069,Almax. ZUHP. Kędzior J.,"{'postalcode': '39-300', 'streetaddress': 'Woj...",17 788 61 16,https://www.biznesfinder.pl/podkarpackie/metal...,None,almax@almax.mielec.pl,"{'latitude': '50.3052', 'longitude': '21.46521'}",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,50.3052,21.46521,"[84790, 89967]",0.000000
203065,7071,Małgorzata Patryas Wynajem sprzętu budowlanego,"{'streetaddress': 'Motylewska 24', 'addressreg...",67 212 48 37,https://www.biznesfinder.pl/wielkopolskie/budo...,None,wynajemsprzetubudowlanego@neostrada.pl,"{'longitude': '16.756554627451', 'latitude': '...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,53.1313001372549,16.756554627451,"[85865, 89969]",0.000000
203076,7082,"A/ =jsk= Handel-usługi Jarosław Kowalski, B/ =...","{'addresslocality': 'Warszawa', 'postalcode': ...",None,https://www.biznesfinder.pl/warszawa-a-jsk-han...,None,None,"{'latitude': '52.2293207188778', 'longitude': ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,52.2293207188778,21.0477241177767,"[87004, 88301]",0.000000


In [37]:
print(data.iloc[12])
print(data.iloc[10272])

row_id                                                                      27
name                                                              Apollo-Optik
address                      {'addressregion': 'Rheinland-Pfalz', 'addressc...
telephone                                                  (06131)&nbsp;693876
page_url                     https://www.11880.com/branchenbuch/mainz-am-rh...
aggregaterating              {'reviewcount': '12', 'bestrating': '5', 'wors...
email                                                 service@apollo-optik.com
geo                            {'latitude': '50,001', 'longitude': '8,270224'}
url                                     https://www.apollo.de/store-7053/mainz
image                        https://cdn.11880.com/apollo-optik_17032642_mw...
openinghoursspecification    [{'opens': '09:30', 'dayofweek': 'Samstag', 'c...
faxnumber                                                                 None
review                       [{'description': 'Zwar 